In [184]:
import requests
import numpy as np
import json
import os
import time
import pandas as pd

API_KEY = 'RGAPI-19f1110a-6e1f-40a9-be6d-3e44f385384c'
# Define the region and platform
REGION = 'kr'
PLATFORM = 'asia'

In [218]:
def get_top_challengers(top = 5):
    url = f"https://{REGION}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"
    params = {'api_key': API_KEY}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        leaderboard = response.json()
        top_challengers = [entry['summonerId'] for entry in leaderboard['entries'][:top]]
        return top_challengers
    else:
        raise Exception("API request failed with status code: " + str(response.status_code))
        
def get_puuid(summoner_id):
    url = f"https://{REGION}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
    params = {'api_key': API_KEY}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        response_json = response.json()
        return response_json['puuid']

def get_match_ids(puuid, count=10, queue=420):
    params = {
        'api_key': API_KEY,
        'queue': queue,
        'start': 0,  # Start at the beginning of the match list
        'count': count  # Number of matches to retrieve
    }
    matchlist_url = f"https://{PLATFORM}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    try:
        matchlist_response = requests.get(matchlist_url, params=params)
        matchlist_response.raise_for_status()
        matchlist = matchlist_response.json()
        # Assuming the API returns the matches in reverse chronological order, reverse the list to get the earliest games
        earliest_match_ids = matchlist[::-1]
        return earliest_match_ids[:count]  # Return only the count of earliest games
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    return []

def get_match_data(match_id):
    url = f"https://{PLATFORM}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    params = {'api_key': API_KEY}
    response = requests.get(url, params=params)
    match_data = response.json()
    return match_data

In [369]:
def normalize_matrix(matrix):
    for position1 in matrix:
        for champ1 in matrix[position1]:
            for position2 in matrix[position1][champ1]:
                total_matches = sum(matrix[position1][champ1][position2].values())
                for champ2 in matrix[position1][champ1][position2]:
                    if total_matches > 0:
                        matrix[position1][champ1][position2][champ2] /= total_matches

def calculate_synergy_and_counter(match_data):
    
    synergy_matrix = {}
    counter_matrix = {}

    for match in match_data:
        winning_team_champs = match['winning_team_champs']
        losing_team_champs = match['losing_team_champs']
        # Update synergy matrix for the winning team
        for i, (winner_i, position_i) in enumerate(winning_team_champs):
            for j, (winner_j, position_j) in enumerate(winning_team_champs[i+1:], start=i+1):
                synergy_matrix.setdefault(position_i, {}).setdefault(winner_i, {}).setdefault(position_j, {}).setdefault(winner_j, 0)
                synergy_matrix[position_i][winner_i][position_j][winner_j] += 1
                # Assuming symmetry
                synergy_matrix.setdefault(position_j, {}).setdefault(winner_j, {}).setdefault(position_i, {}).setdefault(winner_i, 0)
                synergy_matrix[position_j][winner_j][position_i][winner_i] += 1
        
        # Update counter matrix
        for (winner, position_winner) in winning_team_champs:
            for (loser, position_loser) in losing_team_champs:
                counter_matrix.setdefault(position_winner, {}).setdefault(winner, {}).setdefault(position_loser, {}).setdefault(loser, 0)
                counter_matrix[position_winner][winner][position_loser][loser] += 1

    # Normalize the synergy and counter matrices
    normalize_matrix(synergy_matrix)
    normalize_matrix(counter_matrix)

    return synergy_matrix, counter_matrix


In [328]:
import time
from tqdm import tqdm

def get_matches_of_top_players():
    top_players = get_top_challengers(top=20)
    top_player_puuids = [get_puuid(x) for x in top_players]
    all_match_ids = []
    for puuid in top_player_puuids:
        earliest_match_ids = get_match_ids(puuid, count=10)
        all_match_ids.extend(earliest_match_ids)
        
    all_champs = []
    for match_id in tqdm(all_match_ids, desc="Getting match detail", unit="MatchID"):
        match_data = get_match_data(match_id)
        # Extract winning and losing team champion names with their positions
        winning_team_champs = [(participant['championName'], participant['teamPosition']) 
                               for participant in match_data["info"]['participants'] 
                               if participant['win']]
        losing_team_champs = [(participant['championName'], participant['teamPosition']) 
                              for participant in match_data["info"]['participants'] 
                              if not participant['win']]
        if (winning_team_champs, losing_team_champs) not in all_champs:
            all_champs.append((winning_team_champs, losing_team_champs))
        time.sleep(1)  # Be careful with sleep to avoid hitting rate limits
        
    print("The number of unique matches: ", len(all_champs))
    return all_champs

In [388]:
def save_and_load_match_data(filename='all_match_data.json'):
    
    # Check if the file exists
    if os.path.exists(filename):
        # Read the data from the file
        with open(filename, 'r') as file:
            all_match_data = json.load(file)
        print(f'Data loaded from {filename}')
    else:
        # Get the data by calling the function
        all_match_data = get_matches_of_top_players()

        # Convert your data to a list of dictionaries if it's not already in that format
        all_match_data_dicts = [
            {
                'winning_team_champs': winning_team_champs,
                'losing_team_champs': losing_team_champs
            }
            for winning_team_champs, losing_team_champs in all_match_data
        ]

        # Save the data to a file
        with open(filename, 'w') as file:
            json.dump(all_match_data_dicts, file, indent=4)
        print(f'Data saved to {filename}')
        
    return all_match_data

In [ ]:
all_match_data = save_and_load_match_data()

In [370]:
synergy_matrix, counter_matrix = calculate_synergy_and_counter(all_match_data)

In [418]:
def recommend_champions(synergy_matrix, counter_matrix, ally_champions, enemy_champions, position):
    # Extract just the champion names from ally_champions and enemy_champions
    ally_champion_names = [champ for champ, pos in ally_champions]
    enemy_champion_names = [champ for champ, pos in enemy_champions]

    # Calculate synergy scores for potential champions
    # Exclude already picked champions by the ally team and enemy team
    potential_champions = set(synergy_matrix[position].keys()) - set(ally_champion_names) - set(enemy_champion_names)
    
    synergy_scores = {}
    for champ in potential_champions:
        synergy_score = 0
        for ally, ally_pos in ally_champions:
            # Check if the ally champion is not the same as the potential champion
            if ally != champ:
                synergy_score += synergy_matrix.get(ally_pos, {}).get(ally, {}).get(position, {}).get(champ, 0)
        synergy_scores[champ] = synergy_score

    # Calculate counter scores for potential champions
    counter_scores = {}
    for champ in potential_champions:
        counter_score = 0
        for enemy, enemy_pos in enemy_champions:
            counter_score += counter_matrix.get(position, {}).get(champ, {}).get(enemy_pos, {}).get(enemy, 0)
        counter_scores[champ] = counter_score

    # Combine scores and create a list of tuples
    combined_scores = [(champ, synergy_scores[champ], counter_scores[champ], synergy_scores[champ] + counter_scores[champ]) for champ in potential_champions]

    # Create a DataFrame
    df = pd.DataFrame(combined_scores, columns=['Champion', 'Synergy Score', 'Counter Score', 'Total Score'])
    df['Harmonic Score'] = 2*(df['Synergy Score']*df['Counter Score'])/df['Total Score']
    # Sort the DataFrame by the best overall score
    df_sorted = df.sort_values(by='Harmonic Score', ascending=False)
    df_sorted = df_sorted[df_sorted['Total Score'] != 0].reset_index(drop=True)
    df_sorted.drop(columns = ['Total Score'], inplace=True)
    
    best_synergy = df_sorted[df_sorted['Synergy Score'] == max(df_sorted['Synergy Score'])]
    best_counter = df_sorted[df_sorted['Counter Score'] == max(df_sorted['Counter Score'])]
    
    return df_sorted, best_synergy, best_counter

In [473]:
ally_champions = [("JarvanIV", "JUNGLE"), ("Jinx", "BOTTOM")]
enemy_champions = [("Taliyah", "JUNGLE"), ("Aatrox", "TOP"), ('Sylas',"MIDDLE")]
recommended_champions, best_synergy, best_counter = recommend_champions(synergy_matrix, counter_matrix, \
                                                                        ally_champions, enemy_champions, "UTILITY")
recommended_champions

,Champion,Synergy Score,Counter Score,Harmonic Score
0,Renata,0.244870,0.550000,0.338869
1,TahmKench,0.235294,0.333333,0.275862
2,Rakan,0.269494,0.181818,0.217140
3,Nautilus,0.269494,0.176471,0.213280
4,Rell,0.268126,0.157895,0.198749
5,Milio,0.164159,0.250000,0.198183
6,Alistar,0.128591,0.227273,0.164250
7,Galio,0.082079,0.500000,0.141011
8,Pyke,0.046512,0.285714,0.080000
9,Blitzcrank,0.046512,0.111111,0.065574


In [474]:
best_synergy

,Champion,Synergy Score,Counter Score,Harmonic Score
2,Rakan,0.269494,0.181818,0.21714
3,Nautilus,0.269494,0.176471,0.21328


In [475]:
best_counter

,Champion,Synergy Score,Counter Score,Harmonic Score
10,Lulu,0.023256,1.0,0.045455
